<a href="https://colab.research.google.com/github/yuji-sgs/web-optimization/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2章: 確率的プログラミング: コンピュータの助けを借りる